## Importing all the required packages

In [16]:
from jmetal.algorithm.multiobjective.spea2 import SPEA2
from jmetal.operator import PolynomialMutation
from jmetal.problem import ZDT1
from jmetal.util.termination_criterion import StoppingByEvaluations
from jmetal.core.problem import FloatProblem
from jmetal.core.solution import FloatSolution


In [17]:
import copy
import random
from typing import List

from jmetal.core.operator import Crossover
from jmetal.core.solution import Solution, FloatSolution, BinarySolution, PermutationSolution, IntegerSolution,CompositeSolution
from jmetal.util.ckecking import Check

"""
.. module:: crossover
   :platform: Unix, Windows
   :synopsis: Module implementing crossover operators.

.. moduleauthor:: Antonio J. Nebro <antonio@lcc.uma.es>, Antonio Benítez-Hidalgo <antonio.b@uma.es>
"""
class SBXCrossover(Crossover[FloatSolution, FloatSolution]):
    __EPS = 1.0e-14

    def __init__(self, probability: float, distribution_index: float = 20.0):
        super(SBXCrossover, self).__init__(probability=probability)
        self.distribution_index = distribution_index
        if distribution_index < 0:
            raise Exception("The distribution index is negative: " + str(distribution_index))

    def execute(self, parents: List[FloatSolution]) -> List[FloatSolution]:
        Check.that(issubclass(type(parents[0]), FloatSolution), "Solution type invalid: " + str(type(parents[0])))
        Check.that(issubclass(type(parents[1]), FloatSolution), "Solution type invalid")
        Check.that(len(parents) == 2, 'The number of parents is not two: {}'.format(len(parents)))

        offspring = [copy.deepcopy(parents[0]), copy.deepcopy(parents[1])]
        rand = random.random()

        if rand <= self.probability:
            for i in range(parents[0].number_of_variables):
                value_x1, value_x2 = parents[0].variables[i], parents[1].variables[i]

                if random.random() <= 0.5:
                    if abs(value_x1 - value_x2) > self.__EPS:
                        if value_x1 < value_x2:
                            y1, y2 = value_x1, value_x2
                        else:
                            y1, y2 = value_x2, value_x1

                        lower_bound, upper_bound = parents[0].lower_bound[i], parents[1].upper_bound[i]

                        beta = 1.0 + (2.0 * (y1 - lower_bound) / (y2 - y1))
                        alpha = 2.0 - pow(beta, -(self.distribution_index + 1.0))

                        rand = random.random()
                        if rand <= (1.0 / alpha):
                            betaq = pow(rand * alpha, (1.0 / (self.distribution_index + 1.0)))
                        else:
                            betaq = pow(1.0 / (2.0 - rand * alpha), 1.0 / (self.distribution_index + 1.0))

                        c1 = 0.5 * (y1 + y2 - betaq * (y2 - y1))
                        beta = 1.0 + (2.0 * (upper_bound - y2) / (y2 - y1))
                        alpha = 2.0 - pow(beta, -(self.distribution_index + 1.0))

                        if rand <= (1.0 / alpha):
                            betaq = pow((rand * alpha), (1.0 / (self.distribution_index + 1.0)))
                        else:
                            betaq = pow(1.0 / (2.0 - rand * alpha), 1.0 / (self.distribution_index + 1.0))

                        c2 = 0.5 * (y1 + y2 + betaq * (y2 - y1))
                        
                        if  c1 < lower_bound:
                            c1 = lower_bound
                        if c2 < lower_bound:
                            c2 = lower_bound
                        if c1 > upper_bound:
                            c1 = upper_bound
                        if c2 > upper_bound:
                            c2 = upper_bound

                        if random.random() <= 0.5:
                            offspring[0].variables[i] = c2
                            offspring[1].variables[i] = c1
                        else:
                            offspring[0].variables[i] = c1
                            offspring[1].variables[i] = c2
                    else:
                        offspring[0].variables[i] = value_x1
                        offspring[1].variables[i] = value_x2
                else:
                    offspring[0].variables[i] = value_x1
                    offspring[1].variables[i] = value_x2
        return offspring

    def get_number_of_parents(self) -> int:
        return 2

    def get_number_of_children(self) -> int:
        return 2

    def get_name(self) -> str:
        return 'SBX crossover'


## Defining the problem:
    
## obj-1: increasing the profit

## obj-2:decreasing the cvar

In [18]:

class o(FloatProblem):
    profit_values=[]
    cvar_values=[]
    def __init__(self):
        super(o, self).__init__()
        self.number_of_variables = 4
        self.number_of_objectives = 2
        self.number_of_constraints = 1

        self.obj_directions = [self.MINIMIZE,self.MINIMIZE]
        self.obj_labels = ['f(x)', 'f(y)']

        # Constraints
        self.lower_bound = [0,0,0,0.15]
        self.upper_bound = [0.5,0.5,0.15,0.5]
        
    #['home_lons','auto_loans','card_loans','bussiness_loans']
    
    def evaluate(self, solution: FloatSolution) -> FloatSolution:
        #[8.96,7.75,8.21,4.58]
        # profit percentile
        profit=[0.0896,0.075,0.0821,0.0458]
        
        # budget
        max_value=1000
        #[29.2,20.2,20.7,39.5]
        # cvar percentile
        cvar=[0.292,0.202,0.207,0.395]
        
        profit_score=0
        cvar_score=0
        
        q=0
#         print(solution)
        sum_of_variables=sum(solution.variables)
        for i in solution.variables:
            i=i/(sum_of_variables)
            profit_score+=((max_value*i)*profit[q])
            cvar_score+=(cvar[q]*(i*max_value))**2   
            q+=1
        
        cvar_score=((0.25*cvar_score)**(0.5))
        o.profit_values.append(profit_score)
        o.cvar_values.append(cvar_score)
#         print("cvar---->",cvar)
#         print("profit---->",profit)
  
        solution.objectives[0]=1/profit_score # maximising the return
        solution.objectives[1]=cvar_score     # minimising the cvar
        
        self.__evaluate__constraints(solution) # constraints
        return solution
    def get_name(self):
        return 'Tanaka'
    def __evaluate__constraints(self,sol):
        sol.constraints.append(1/1000*(sum(sol.variables)-1)) 
        
        
        
       

In [19]:
import time
max_evaluations = 3000
problem=o()
start_time=time.time()
algorithm = SPEA2(
    problem=problem,
    population_size=40,
    offspring_population_size=40,
    mutation=PolynomialMutation(probability=1.0 / problem.number_of_variables, distribution_index=20),
    crossover=SBXCrossover(probability=1.0, distribution_index=20),
    termination_criterion=StoppingByEvaluations(max_evaluations)
)

algorithm.run()
solutions = algorithm.get_result()
end_time=time.time()
print(end_time-start_time)

103.29037690162659


In [28]:
# from jmetal.lab.visualization.plotting import Plot
# from jmetal.util.solution import get_non_dominated_solutions

# front = get_non_dominated_solutions(solutions)
# for i in front:
#     print(i.variables)

In [29]:
data={"home_loans":[],"automobile":[],"card_loans":[],"bussiness_loans":[],"profit":[],"cvar":[],"total_budget_used":[]}
for i in solutions:
    variables=list(i.variables)
    sum_of_variables=sum(variables)
    for i in range(len(variables)):
        variables[i]=variables[i]/sum_of_variables
    sum_of_variables=sum(variables)
    o3=0
    ##  change
    l=[0.0896,0.075,0.0821,0.0458]
    cs=[0.292,0.202,0.207,0.395]
    o1=0
    o4=0
    data['home_loans'].append(variables[0]/sum_of_variables)
    data['automobile'].append(variables[1]/sum_of_variables)
    data['card_loans'].append(variables[2]/sum_of_variables)
    data['bussiness_loans'].append(variables[3]/sum_of_variables)
    data['total_budget_used'].append(sum_of_variables)
    for j in variables:
        o3+=(j*1000)*l[o1]
        o4+=((j*1000)*cs[o1])**2
        o1+=1
    o4=((0.25*o4)**(0.5))
    data["profit"].append(o3)
    data["cvar"].append(o4)
    print(o3,o4)
    print()

78.97589003318856 82.66769385486401

78.83564006182439 80.79431448710055

77.27187194736483 70.55454086519163

78.69982938711865 78.98922723073142

78.20973634201371 74.21855257745582

76.15101925791278 68.20589335768723

78.48561168702405 76.51131106724341

78.73842923537828 79.44461202431327

77.48812898227827 71.15302989246334

77.61860926419445 71.55482001574875

78.6636926414356 78.51247878097308

78.88550553706835 81.3845767081847

77.86398410926249 72.3185334675267

77.74300694985229 71.9723868058367

78.36913129781802 75.35106265584157

78.52473876496485 76.92494963341971

78.2934323611067 74.67139404596976

78.33147996190075 75.0095424999307

78.79629290848173 80.18251329462804

78.77027556614497 79.84747079076696

76.71844746405165 69.24941944627626

77.061520078502 70.03317653400713

76.87211278645958 69.70651067525823

74.74678001686301 67.15953893832615

78.52475818635558 77.26363579461213

78.5829936434909 77.57708143627134

78.43649563790494 76.0308096246213

78.40825885

In [30]:
for i,j in data.items():
    print(i,len(j))

home_loans 40
automobile 40
card_loans 40
bussiness_loans 40
profit 40
cvar 40
total_budget_used 40


In [31]:
import pandas as pd
data1=pd.DataFrame(data)
data1

,home_loans,automobile,card_loans,bussiness_loans,profit,cvar,total_budget_used
0,0.498866,0.201813,0.149659,0.149662,78.975890,82.667694,1.0
1,0.482030,0.228525,0.144623,0.144823,78.835640,80.794314,1.0
2,0.342463,0.410680,0.123420,0.123437,77.271872,70.554541,1.0
3,0.464591,0.256527,0.139398,0.139484,78.699829,78.989227,1.0
4,0.406792,0.346210,0.123497,0.123502,78.209736,74.218553,1.0
5,0.284208,0.444456,0.135664,0.135672,76.151019,68.205893,1.0
6,0.437446,0.300037,0.131250,0.131266,78.485612,76.511311,1.0
7,0.469175,0.249268,0.140770,0.140787,78.738429,79.444612,1.0
8,0.353931,0.403634,0.121208,0.121227,77.488129,71.153030,1.0
9,0.361485,0.397890,0.120308,0.120317,77.618609,71.554820,1.0


In [32]:
# maximum profit
data1[data1['profit']==data1['profit'].max()]

,home_loans,automobile,card_loans,bussiness_loans,profit,cvar,total_budget_used
0,0.498866,0.201813,0.149659,0.149662,78.97589,82.667694,1.0


In [33]:
#minimum cvar
data1[data1['cvar']==data1['cvar'].min()]

,home_loans,automobile,card_loans,bussiness_loans,profit,cvar,total_budget_used
23,0.20926,0.49136,0.149683,0.149697,74.74678,67.159539,1.0


In [36]:
#Inverted generation distance
pareto_profit=list(data1['profit'])
pareto_cvar=list(data1['cvar'])
min_values=[]
for i in range(len(pareto_profit)):
    m=-1
    for j in range(3000):
        euclidean_distance=((pareto_profit[i]-o.profit_values[j])**2+(pareto_cvar[i]-o.cvar_values[j])**2)**0.5
        if(m==-1):
            m=euclidean_distance
        elif(euclidean_distance==0):
            continue
        else:
            m=min(m,euclidean_distance)
    min_values.append(m)
print((sum(min_values)**0.5)/len(min_values))

0.011736170215427182


In [37]:
#Schott’s spacing metric
min_diff=[]
for i in range(len(o.profit_values)):
    m=-1
    for j in range(40):
        euclidean_distance=((pareto_profit[j]-o.profit_values[i])**2+(pareto_cvar[j]-o.cvar_values[i])**2)**0.5
        if(m==-1):
            m=euclidean_distance
        else:
            m=min(m,euclidean_distance)
    min_diff.append(m)
avg=sum(min_diff)/len(min_diff)
n=0
for i in min_diff:
    n+=abs(i-avg)
print(((1/3000)*n)**0.5)

1.3371152095426158


In [38]:
min_values


[0.00030550593852658465,
 7.408640692594675e-05,
 0.00495710186575321,
 0.0029416806718562095,
 0.01363495710855311,
 1.860625098011403e-06,
 2.585126268310912e-05,
 3.2384028906312266e-06,
 0.0008025141848773215,
 0.027577524156948387,
 0.00014316048962136795,
 0.0020455955285882885,
 0.0003333670484380862,
 0.017556923146828676,
 0.012753264908116596,
 0.003278340670348426,
 1.2634449451658873e-05,
 0.00345748945578645,
 1.0502129450727137e-06,
 1.5415523860785088e-06,
 0.010107748988776361,
 0.0014471555043075626,
 0.019859054105498968,
 1.703082946152925e-05,
 0.027551167372674244,
 2.1939871956217202e-05,
 0.0005742319091745696,
 4.2763892820631723e-07,
 0.00044447475359181974,
 9.20119544666041e-06,
 0.008805937851422217,
 0.008052524097554718,
 0.00547548007216217,
 0.008517245962957344,
 0.0006995660841767609,
 0.014103750935921724,
 0.008072513746208652,
 0.0008714542821362948,
 0.01570866363060416,
 0.00013304920118471065]